# Task 2: End-to-End ML Pipeline for Customer Churn Prediction

## 🎯 Objective
Build a reusable ML pipeline using Scikit-learn to predict customer churn in a telecom dataset.

In [ ]:
# 📦 Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib

In [ ]:
# 📂 Load dataset
df = pd.read_csv('/mnt/data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df = df.drop(columns=['customerID'])
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
print(df.head())

In [ ]:
# ✂️ Feature-target split
X = df.drop('Churn', axis=1)
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# 🔍 Preprocessing setup
numeric_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
categorical_features = [col for col in X.columns if col not in numeric_features]

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

In [ ]:
# 🧪 Pipeline definition
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('clf', LogisticRegression(max_iter=1000))
])

In [ ]:
# 🔍 GridSearchCV
param_grid = [
    {
        'clf': [LogisticRegression(max_iter=1000)],
        'clf__C': [0.01, 0.1, 1.0, 10]
    },
    {
        'clf': [RandomForestClassifier()],
        'clf__n_estimators': [50, 100],
        'clf__max_depth': [5, 10]
    }
]

grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)

In [ ]:
# 📊 Evaluate
y_pred = grid_search.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
# 💾 Save model
joblib.dump(grid_search.best_estimator_, '/mnt/data/telco_churn_pipeline.joblib')
print("✅ Pipeline exported successfully.")